# Homework 1
### Nicholas Colonna

"I pledge my honor that I have abided by the Stevens Honor System." -Nicholas Colonna

First, we will import the packages that are used in our assignment.

In [1]:
import pandas as pd
import pylab as pl
import numpy as np
import re
import os
import statsmodels.api as sm

Next, we modify our working directory to our file location. Once there we read in our csv data and safe it as variable df

In [2]:
#Modify the path:
path ="C:/Users/colon/Desktop/Fall 2018/QF 301 Advanced Time Series Analysis/Homework1"
os.chdir(path)

df = pd.read_csv("./data/credit-training.csv")

1. Generate a table with number of null variables by variable

This function was taken from a previous lab where we did the same thing. Essentially, the function takes in our dataframe and counts all the null values for each variable. The output will show the number of nulls per variable, which are indicated under the 'false' column of the output. It can be seen that only MonthlyIncome and NumberOfDependents have null values.

In [3]:
def print_null_freq(df):
    #for a given DataFrame, calculates how many values for each variable is null and prints the resulting table to stdout
    df_lng = pd.melt(df)
    null_variables = df_lng.value.isnull()
    return pd.crosstab(df_lng.variable, null_variables)

print (print_null_freq(df))

value                                  False  True 
variable                                           
DebtRatio                             150000      0
MonthlyIncome                         120269  29731
NumberOfDependents                    146076   3924
NumberOfOpenCreditLinesAndLoans       150000      0
NumberOfTime30-59DaysPastDueNotWorse  150000      0
NumberOfTime60-89DaysPastDueNotWorse  150000      0
NumberOfTimes90DaysLate               150000      0
NumberRealEstateLoansOrLines          150000      0
RevolvingUtilizationOfUnsecuredLines  150000      0
SeriousDlqin2yrs                      150000      0
age                                   150000      0


2. Convert column names from camelCase into snake_case

This function was taken from a previous lab where we did the same thing. We call the function for each variable name in df. The function locates the capital letters, adds an underscore before it, then changes the capital letter to lower case. You can see the changes below.

In [4]:
def camel_to_snake(column_name):
    #converts a string that is camelCase into snake_case
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', column_name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

print(df.columns)
df.columns = [camel_to_snake(col) for col in df.columns]
print(df.columns)

Index(['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')
Index(['serious_dlqin2yrs', 'revolving_utilization_of_unsecured_lines', 'age',
       'number_of_time30-59_days_past_due_not_worse', 'debt_ratio',
       'monthly_income', 'number_of_open_credit_lines_and_loans',
       'number_of_times90_days_late', 'number_real_estate_loans_or_lines',
       'number_of_time60-89_days_past_due_not_worse', 'number_of_dependents'],
      dtype='object')


3. After exploring the dataset, count the number of cases with all the following characteristics as a single query:
     -people 35 or older
     -who have no been delinquent in the past 2 years
     -who have less than 10 open credit lines/loans
     
First, I created a filter based off of the characteristics above. All that was needed was to reference the given characteristic and set some constraint to check for. After that, since we were only concerned with the number of cases, we print the length of the filter just created.

In [5]:
filter1 = (df.age >= 35) & (df.serious_dlqin2yrs == 0) & (df.number_of_open_credit_lines_and_loans < 10)
print()
print(len(df[filter1]))


76151


4. Repeat the exercise with all these restrictions as a single query:
     -people who have been delinquent in the past 2 years
     -are in the 90th percentile for monthly_income
     
Similar to the problem above, we created a filter for the characteristics we are looking for. We utilize '.quantile' as a help when we filter for monthly_income percentile. Since we are only concerned with the number of cases, we print the length of the filter just created.

In [6]:
filter2 = (df.serious_dlqin2yrs == 1) & (df.monthly_income >= df.monthly_income.quantile(.90))
print()
print(len(df[filter2]))


539


5. Build and describe a forecasting model of the variable SeriousDlqin2yrs using logistic regression.

My first idea was to run a regression on all variables and see which were significant. To do this, I needed to remove all the NAs from the data. After that I created my variables for the regression, y for delinquincies and x for every other variable. After that, I was able to run a logistic regression on the data and produce results. With this data, we are able to analyze the significance of each variable

In [7]:
df = df.dropna()
y = df.serious_dlqin2yrs
x = df[['revolving_utilization_of_unsecured_lines','age','number_of_time30-59_days_past_due_not_worse','debt_ratio','monthly_income','number_of_open_credit_lines_and_loans','number_of_times90_days_late','number_real_estate_loans_or_lines','number_of_time60-89_days_past_due_not_worse','number_of_dependents']].copy()
logit_model = sm.Logit(y,x)
result = logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.237361
         Iterations 7
                                       Results: Logit
Model:                        Logit                     Pseudo R-squared:         0.059     
Dependent Variable:           serious_dlqin2yrs         AIC:                      57114.2447
Date:                         2018-09-26 12:02          BIC:                      57211.2196
No. Observations:             120269                    Log-Likelihood:           -28547.   
Df Model:                     9                         LL-Null:                  -30345.   
Df Residuals:                 120259                    LLR p-value:              0.0000    
Converged:                    1.0000                    Scale:                    1.0000    
No. Iterations:               7.0000                                                        
--------------------------------------------------------------------------------------------
       

As seen from the results above, all of the variables, except for 'revolving_utilization_of_unsecured_lines', are significant (p<.05). Therefore, we rerun the regression with the same steps, but we remove that variable. The results of this regression show that all variables are significant (p<.05), therefore, this model seems to be a good forecaster for serious_dlqin2yrs.

In [8]:
#revolving_utilization_of_unsecured_lines is not a significant variable, drop from regression
x = df[['age','number_of_time30-59_days_past_due_not_worse','debt_ratio','monthly_income','number_of_open_credit_lines_and_loans','number_of_times90_days_late','number_real_estate_loans_or_lines','number_of_time60-89_days_past_due_not_worse','number_of_dependents']].copy()
logit_model = sm.Logit(y,x)
result = logit_model.fit()
print(result.summary2())
#All variables in this regression are significant, therefore it can be a useful forecaster

Optimization terminated successfully.
         Current function value: 0.237367
         Iterations 7
                                       Results: Logit
Model:                        Logit                     Pseudo R-squared:         0.059     
Dependent Variable:           serious_dlqin2yrs         AIC:                      57113.7081
Date:                         2018-09-26 12:02          BIC:                      57200.9854
No. Observations:             120269                    Log-Likelihood:           -28548.   
Df Model:                     8                         LL-Null:                  -30345.   
Df Residuals:                 120260                    LLR p-value:              0.0000    
Converged:                    1.0000                    Scale:                    1.0000    
No. Iterations:               7.0000                                                        
--------------------------------------------------------------------------------------------
       